In [16]:
# imports
import datetime as dt
import json
import random
from itertools import combinations_with_replacement

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

from game_lists_site.models import BenchmarkUserCBR, Game, System, User, UserGame, db
from game_lists_site.utils.utils import (
    days_delta,
    get_cbr_for_game,
    get_normalized_playtimes,
    normalize_dict,
)

db.rollback()


In [17]:
# get users
users = []
for user in User.select(User.id, User.username):
    game_with_score_count = (
        UserGame.select(UserGame.id)
        .where((UserGame.user == user) & (UserGame.score > 0))
        .count()
    )
    if game_with_score_count > 1:
        users.append(user)
print(users)

[<User: 76561198125290350>, <User: 76561198091812571>, <User: 76561198067514875>, <User: 76561198094109207>, <User: 76561198083927294>, <User: 76561198026681120>, <User: 76561198394079733>]


In [18]:
# cbr for user
def reset_benchmark_cbr_for_user(users):
    #users = User.select().where(User.last_benchmark_cbr_update_time != None)
    system, _ = System.get_or_create(key="GameCBR")
    system.date_time_value = None
    system.save()
    for user in users:
        user.last_benchmark_cbr_update_time = None
        user.save()


def check_result(result_games: list, check_games: list):
    check_game_len = len(check_games)
    return (
        len(set(result_games[: len(check_games)]).intersection(check_games))
        / check_game_len
    )


def get_benchmark_cbr_for_user(
    user, result_count=-1, min_player_count=28, cbr_for_game_result_count=3
):
    # print(user.last_benchmark_cbr_update_time)
    if (
        not user.last_benchmark_cbr_update_time
        or days_delta(user.last_benchmark_cbr_update_time) >= 7
    ):
        played_user_games = UserGame.select(
            UserGame.game, UserGame.last_played, UserGame.score
        ).where((UserGame.user == user) & (UserGame.playtime > 0))
        last_played = np.quantile([ug.last_played for ug in played_user_games], 0.8)
        played_user_games = played_user_games.where(UserGame.last_played < last_played)
        played_games = [ug.game for ug in played_user_games]
        user_games_with_score = played_user_games.where(UserGame.score > 0)
        games_with_score = [ug.game for ug in user_games_with_score]
        result = {}
        for user_game, game_cbr_result in zip(
            user_games_with_score,
            [
                get_cbr_for_game(g, cbr_for_game_result_count, min_player_count)
                for g in games_with_score
            ],
        ):
            if game_cbr_result:
                for sim_game in game_cbr_result:
                    if sim_game not in played_games and sim_game.rating >= 7:
                        if sim_game.id not in result:
                            result[sim_game.id] = (
                                user_game.score * game_cbr_result[sim_game]
                            )
                        else:
                            result[sim_game.id] += (
                                user_game.score * game_cbr_result[sim_game]
                            )
        benchmark_user_cbr, _ = BenchmarkUserCBR.get_or_create(user=user)
        benchmark_user_cbr.data = json.dumps(
            normalize_dict(
                dict(sorted(result.items(), key=lambda x: x[1], reverse=True))
            )
        )
        benchmark_user_cbr.save()
        benchmark_user_cbr.save()
        user.last_benchmark_cbr_update_time = dt.datetime.now()
        user.save()
    data = {
        Game.get_by_id(game_id): value
        for game_id, value in json.loads(
            BenchmarkUserCBR.get_or_none(BenchmarkUserCBR.user == user).data
        ).items()
    }
    if result_count != -1:
        return dict(list(data.items())[:result_count])
    else:
        return dict(list(data.items()))


def train_cbr_for_user(users, current_accuracy):
    best_accuracy = current_accuracy
    best_cbr_for_game_result = 1
    best_min_player_count = 1
    comb = combinations_with_replacement(range(1, 50, 1), 2)
    for a0, a1 in comb:
        reset_benchmark_cbr_for_user(users)
        min_player_count = a0
        cbr_for_game_result_count = a1
        cbr_accuracy = []
        for user in users:
            played_user_games = UserGame.select(
                UserGame.game, UserGame.last_played
            ).where((UserGame.user == user) & (UserGame.playtime > 0))
            last_played = np.quantile([ug.last_played for ug in played_user_games], 0.8)
            input_user_games = played_user_games.where(
                UserGame.last_played < last_played
            )
            check_user_games = played_user_games.where(
                UserGame.last_played >= last_played
            )
            played_games = [ug.game for ug in input_user_games]
            check_games = [ug.game for ug in check_user_games]
            cbr_result = get_benchmark_cbr_for_user(
                user, -1, min_player_count, cbr_for_game_result_count
            )
            cbr_accuracy.append(check_result(list(cbr_result.keys()), check_games))
        current_accuracy = np.mean(cbr_accuracy)
        print(current_accuracy, min_player_count, cbr_for_game_result_count)
        best_min_player_count = min_player_count
        best_cbr_for_game_result = cbr_for_game_result_count
        if  best_accuracy < current_accuracy:
            break
    return (best_min_player_count, best_cbr_for_game_result)


In [19]:
# train_cbr_for_user(users, 0.099)

In [23]:
# mbcf for user
def reset_benchmark_mbcf_for_user():
    system, _ = System.get_or_create(key="BenchmarkUserMBCF")
    system.date_time_value = None
    system.save()
    system, _ = System.get_or_create(key="NormalizedPlaytime")
    system.date_time_value = None
    system.save()

def get_benchmark_mbcf_for_user(
    user,
    max_count=-1,
    min_player_count=10,
    sim_user_count=9,
    zscore_norm=False,
    corrcoef =False,
):
    system, _ = System.get_or_create(key="BenchmarkUserMBCF")
    if not system.date_time_value or days_delta(system.date_time_value) >= 7:
        normalized_playtimes = get_normalized_playtimes(
            min_player_count, True, zscore_norm
        )
        games = list(normalized_playtimes.keys())
        users = {}
        for game in normalized_playtimes.keys():
            for u in normalized_playtimes[game].keys():
                if u in users:
                    users[u] += 1
                else:
                    users[u] = 1
        users = [u for u, game_count in users.items() if game_count >= 10]
        game_vecs = []
        for game in games:
            game_vec = []
            for u in users:
                value = normalized_playtimes[game].get(u)
                game_vec.append(value if value else 0)
            game_vecs.append(game_vec)
        game_vecs = np.array(game_vecs, dtype=np.float32)
        user_vecs = np.flip(np.rot90(game_vecs), 0)
        if corrcoef:
            user_vecs = np.corrcoef(user_vecs)
        else:
            user_vecs = cosine_similarity(user_vecs)
        print(user_vecs)
               
        #users = [
            #user
            #for user in User.select()
            #if len(
                #[
                    #ug
                    #for ug in UserGame.select(
                        #UserGame.user, UserGame.game, UserGame.playtime
                    #).where((UserGame.user == user) & (UserGame.playtime > 0))
                    #if normalized_playtimes.get(ug.game)
                    #and normalized_playtimes.get(ug.game).get(ug.user)
                #]
            #)
            #>= 10
        print(len(games))
        print(len(users))


reset_benchmark_mbcf_for_user()
for user in users[:1]:
    get_benchmark_mbcf_for_user(user, -1, 30)


get_normalized_playtimes 30 True False 376
[[1.0000001e+00 1.5052559e-02 1.8502539e-03 ... 4.4054636e-03
  1.1109752e-02 2.3719645e-04]
 [1.5052559e-02 1.0000000e+00 2.9030506e-02 ... 5.9449848e-02
  1.0158741e-01 2.1890844e-03]
 [1.8502539e-03 2.9030506e-02 9.9999994e-01 ... 1.5310626e-02
  1.3489544e-01 1.1113146e-02]
 ...
 [4.4054636e-03 5.9449848e-02 1.5310626e-02 ... 1.0000000e+00
  1.3359860e-01 5.1696417e-03]
 [1.1109752e-02 1.0158741e-01 1.3489544e-01 ... 1.3359860e-01
  1.0000000e+00 3.8689297e-02]
 [2.3719645e-04 2.1890844e-03 1.1113146e-02 ... 5.1696417e-03
  3.8689297e-02 1.0000001e+00]]
376
896


In [ ]:
# mobcf for user

In [ ]:
# hrs for game

In [ ]:
# benchmark
def check_result(result_games: list, check_games: list):
    check_game_len = len(check_games)
    return len(set(result_games[:len(check_games)]).intersection(check_games)) / check_game_len

cbr_accuracy = []

reset_benchmark_cbr_for_user()

for user in users:
    played_user_games = UserGame.select(UserGame.game, UserGame.last_played).where((UserGame.user == user) & (UserGame.playtime > 0))
    last_played = np.quantile([ug.last_played for ug in played_user_games], 0.8)
    input_user_games = played_user_games.where(UserGame.last_played < last_played)
    check_user_games = played_user_games.where(UserGame.last_played >= last_played)
    played_games = [ug.game for ug in input_user_games]
    check_games = [ug.game for ug in check_user_games]
    cbr_result = get_benchmark_cbr_for_user(user, 36)
    cbr_accuracy.append(check_result(list(cbr_result.keys()), check_games))
print("cbr accuracy:", np.mean(cbr_accuracy))

TypeError: reset_benchmark_cbr_for_user() missing 1 required positional argument: 'users'